## Import Libraries

In [1]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn
import random

random.seed(42)

## Read Data

In [2]:
axial_image = 'MRNet-v1.0/train/axial/0000.npy'
coronal_image = 'MRNet-v1.0/train/coronal/0000.npy'
saggital_image = 'MRNet-v1.0/train/sagittal/0000.npy'


img_array_axial = np.load(axial_image)
img_array_coronal = np.load(coronal_image)
img_array_sagittal = np.load(saggital_image)

In [3]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'


In [4]:
def load_labels(abnormal_path, acl_path, meniscus_path):
    
    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])
    
    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))
    
    return df_lables_combined

In [5]:
# Load Train Labels
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,0000,1,0,0
1,0001,1,1,1
2,0002,1,0,0
3,0003,1,0,1
4,0004,1,0,0
...,...,...,...,...
1125,1125,1,0,1
1126,1126,1,0,1
1127,1127,0,0,0
1128,1128,1,0,0


In [6]:
# Load Valid Labels
df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [7]:
glob.glob('MRNet-v1.0\*')

['MRNet-v1.0\\train',
 'MRNet-v1.0\\train-abnormal.csv',
 'MRNet-v1.0\\train-acl.csv',
 'MRNet-v1.0\\train-meniscus.csv',
 'MRNet-v1.0\\valid',
 'MRNet-v1.0\\valid-abnormal.csv',
 'MRNet-v1.0\\valid-acl.csv',
 'MRNet-v1.0\\valid-meniscus.csv']

In [8]:
# Read Traning Data
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []

    axial_labels = []
    coronal_labels = []
    sagittal_labels = []

    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')
    
    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)
        
        for i in train_data:
#             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                axial_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'coronal' in i:
                coronal_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                coronal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])   
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                sagittal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])

        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images, axial_labels

train_path =  'MRNet-v1.0\\train'
valid_path =  'MRNet-v1.0\\valid'

axial_images_train, coronal_images_train, sagittal_images_train, axial_labels_train = read_data(train_path, df_labels_train)

Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\axial
Loading Complete From :    MRNet-v1.0\train\axial



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\coronal
Loading Complete From :    MRNet-v1.0\train\coronal



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\sagittal
Loading Complete From :    MRNet-v1.0\train\sagittal





In [9]:
# print(np.array(axial_images_train).shape)
print(np.array(coronal_images_train).shape)
print(np.array(sagittal_images_train).shape)
print(np.array(axial_labels_train).shape)

(1130, 256, 256, 16)
(1130, 256, 256, 16)
(1130, 3)


## Modeling

In [10]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


In [11]:
# Two Input Model

def model ():
    visible1 = Input(shape=(256,256,16))
    
    conv11 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(visible1)
    pool11 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv11)
    
    conv12 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv12)
    
    conv13 = Conv2D(filters=16, kernel_size=(5,5), activation='relu')(pool12)
    pool13 = MaxPooling2D(pool_size=(2, 2),strides=(2,2))(conv13)
    flat1 = Flatten()(pool13)
    
    return flat1, visible1
model()

# merge input models
model_1, Visible_1 = model()
model_2, Visible_2 = model()
model_3, Visible_3 = model()

merge = concatenate([model_1, model_2])

hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(5, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
model = Model(inputs=[Visible_1, Visible_2, Visible_3], outputs=output)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_3 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 252, 252, 16  6416        ['input_2[0][0]']                
                                )                                                             

In [12]:
lr = 0.0000001
momentum = 0.8

model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD( learning_rate = lr, momentum = momentum),metrics=["binary_accuracy"])


model.fit(x = [np.array(coronal_images_train), np.array(sagittal_images_train), np.array(axial_images_train)],
          y = np.array(axial_labels_train),
          epochs=3,
          batch_size=32,
          validation_split=0.20)


Epoch 1/3
29/29 [==============================] - 63s 2s/step - loss: 6.2547 - binary_accuracy: 0.4310 - val_loss: 1.1666 - val_binary_accuracy: 0.5472
Epoch 2/3
29/29 [==============================] - 59s 2s/step - loss: 0.8960 - binary_accuracy: 0.6062 - val_loss: 0.8181 - val_binary_accuracy: 0.6165
Epoch 3/3
29/29 [==============================] - 58s 2s/step - loss: 0.7749 - binary_accuracy: 0.6375 - val_loss: 0.7823 - val_binary_accuracy: 0.6357


In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11538599099816661744
xla_global_id: -1
]


In [14]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [15]:
axial_images_valid, coronal_images_valid, sagittal_images_valid, axial_labels_valid = read_data(valid_path, df_labels_valid)

Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\axial
Loading Complete From :    MRNet-v1.0\valid\axial



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\coronal
Loading Complete From :    MRNet-v1.0\valid\coronal



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\sagittal
Loading Complete From :    MRNet-v1.0\valid\sagittal





## Evaluate Model

In [16]:
y_pred = model.predict([ np.array(coronal_images_valid), np.array(sagittal_images_valid), np.array(axial_images_valid)])

4/4 [==============================] - 1s 293ms/step


In [17]:
y_pred

array([[3.1843626e-01, 6.7356402e-01, 9.3699372e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [2.0412137e-01, 9.9684346e-01, 4.4159882e-04],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [7.6296113e-02, 2.1758370e-01, 2.6765238e-02],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [3.2502669e-01, 4.0282407e-01, 2.6563194e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [2.3873854e-01, 3.4870145e-01, 1.6608246e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [3.5950777e-01, 4.2283648e-01, 3.0925071e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [5.0000173e-01, 4.9999872e-01, 5.0000030e-01],
       [4.9173680e-01, 4.9554560e-01, 4.8850048e-01],
       [5.0000173e-01, 4.999

In [18]:
from sklearn.metrics import classification_report
# 
print(classification_report(y_pred.round(), axial_labels_valid))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81        92
           1       0.07      0.57      0.13         7
           2       0.77      0.43      0.55        93

   micro avg       0.60      0.62      0.61       192
   macro avg       0.55      0.61      0.50       192
weighted avg       0.76      0.62      0.66       192
 samples avg       0.49      0.52      0.48       192



C:\Users\Jackson\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jackson\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
from sklearn.metrics import roc_auc_score, accuracy_score
ypreds_abnormal_soft = list(y_pred[:,0])
ypreds_abnormal_hard = list(np.array(y_pred[:,0]).round())
ytrue_abnormal = (np.array(axial_labels_valid)[:,0])
auc_abnormal = roc_auc_score(y_score= ypreds_abnormal_soft, y_true= ytrue_abnormal)
accuracy_abnormal = accuracy_score(y_true= ytrue_abnormal, y_pred= ypreds_abnormal_hard)

ypreds_acl_soft = list(y_pred[:,1])
ypreds_acl_hard = list(np.array(y_pred[:,1]).round())
ytrue_acl = list(np.array(axial_labels_valid)[:,1])
auc_acl = roc_auc_score(y_score= ypreds_acl_soft, y_true= ytrue_acl)
accuracy_acl = accuracy_score(y_true= ytrue_acl, y_pred= ypreds_acl_hard)


ypreds_meniscus_soft = list(y_pred[:,2])
ypreds_meniscus_hard = list(np.array(y_pred[:,2]).round())
ytrue_meniscus = list(np.array(axial_labels_valid)[:,2])
auc_meniscus = roc_auc_score(y_score= ypreds_meniscus_soft, y_true= ytrue_meniscus)
accuracy_meniscus = accuracy_score(y_true= ytrue_meniscus, y_pred= ypreds_meniscus_hard)



logString = "LR: " + str(lr) + "\nMomentum: " + str(momentum) + "\nAbnormal MRI AUC: " + str(auc_abnormal) + "\nACL Injury AUC: " + str(auc_acl) + "\nMeniscus Injury AUC: " + str(auc_meniscus) +"\nAbnormal MRI Accuracy: " + str(accuracy_abnormal) + "\nACL Injury Accuracy: " + str(accuracy_acl) + "\nMeniscus Injury Accuracy: " + str(accuracy_meniscus)

print (logString)

filename = "Logs.txt"
file = open(filename, "a")

file.write(logString)
file.close()



LR: 1e-07
Momentum: 0.8
Abnormal MRI AUC: 0.5785263157894737
ACL Injury AUC: 0.5817901234567902
Meniscus Injury AUC: 0.48359728506787336
Abnormal MRI Accuracy: 0.7083333333333334
ACL Injury Accuracy: 0.5583333333333333
Meniscus Injury Accuracy: 0.4583333333333333


In [20]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

import torch
print(torch.cuda.is_available())

False
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False
True
